In [1]:
import cv2
import numpy as np
import polars as pl

from PIL import Image
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor

from myfunctions import util as mi

In [2]:
# # データフレームの整理

# parquet_path = "../03_Collect_Images/images.parquet"
# df = pl.read_parquet(parquet_path)

# # 条件に基づいてデータをフィルタリング
# filtered_df = (
#     df.filter(pl.col("readable") == True)  # readable列がTrue
#       .filter(pl.col("width") >= 256)      # widthが256以上
#       .filter(pl.col("height") >= 256)     # heightが256以上
#       .filter(pl.col("color_mode").is_in(["RGB", "RGBA"]))  # color_modeがRGBまたはRGBA
#       .filter(pl.col("exclude_same_folder") == False)  # exclude_same_folderがFalse
#       .filter(pl.col("exclude_cross_folder") == "include")  # exclude_cross_folderが'include'
#       .filter(pl.col("too_small") == False)  # too_smallがFalse
#       .filter(~pl.col("path").str.contains("chart"))  # path列に'chart'を含まない
# )

# # 不要な列を削除
# filtered_df = filtered_df.drop(["readable", "exclude_same_folder", "exclude_cross_folder", "too_small", "hash", "__index_level_0__"])

# # 列の順序変更: indexを0列目に移動
# columns = ["index"] + [col for col in filtered_df.columns if col != "index"]
# filtered_df = filtered_df.select(columns)

# # 保存
# output_parquet_path = "./images.parquet"
# filtered_df.write_parquet(output_parquet_path)

# # 結果を確認
# filtered_df.head()

In [3]:
# # ランダムに100行選択
# df = pl.read_parquet("./images.parquet")

# # 画像の色ヒストグラムを並列処理で計算
# def process_row_for_color_histogram(row):
#     """
#     各画像について色ヒストグラムを計算する。

#     Args:
#         row: データフレームの1行（辞書形式）。

#     Returns:
#         色ヒストグラム（NumPy配列）。
#     """
#     # 画像を読み込む（OpenCV形式）
#     image = mi.load_image(row["path"], library="opencv")
#     if image is None:
#         return None

#     # 画像を前処理（明るさ補正とコントラスト強調を統合）
#     preprocessed_image = mi.preprocess_image(image, gamma=1.5, clip_limit=2.0, tile_grid_size=(8, 8))

#     # チャンネル数を確認し、必要に応じて補正
#     if preprocessed_image.ndim == 2:  # グレースケールの場合
#         preprocessed_image = cv2.cvtColor(preprocessed_image, cv2.COLOR_GRAY2BGR)

#     # 画像をリサイズ
#     resized_image = mi.resize_image(preprocessed_image, max_length=128)
#     if resized_image is None:
#         return None

#     # チャンネル数を再確認（念のため）
#     if resized_image.ndim != 3 or resized_image.shape[2] != 3:
#         print(f"Warning: Unexpected channel configuration for image at path: {row['path']}")
#         return None

#     # 色ヒストグラムを計算
#     return mi.calculate_color_histogram(resized_image)


# with ThreadPoolExecutor() as executor:
#     # プログレスバーを設定
#     results = list(tqdm(executor.map(process_row_for_color_histogram, df.to_dicts()), total=len(df)))

# # 結果をDataFrameに追加
# df = df.with_columns(pl.Series("color_histogram", results))

# # PCAで次元削減
# pca_features = mi.pca_reduction(df["color_histogram"].to_list(), n_components=32)
# df = df.with_columns(pl.Series("color_histogram_pca", pca_features.tolist()))

# # 標準化
# normalized_features = mi.normalize_zscore(df["color_histogram_pca"].to_list())
# df = df.with_columns(pl.Series("color_histogram_std", normalized_features.tolist()))

# # Noneやゼロ埋めが含まれているか確認
# none_count = mi.count_none_in_column(df, "color_histogram_std")
# print(f"None values in color_histogram: {none_count}")

# zero_count = mi.count_zeros_in_column(df, "color_histogram_std")
# print(f"Zero values in color_histogram: {zero_count}")

# # 次元が揃っているか確認
# mi.check_dimensions(df, "color_histogram_std")

# # 確認
# print(df.head())

# # 保存
# output_parquet_path = "./temp.parquet"
# df.write_parquet(output_parquet_path)

In [4]:
# # データフレームの読み込み
# df = pl.read_parquet("./temp.parquet")

# # Perceptual Hashを計算する関数
# def process_row_for_phash(row):
#     """
#     各画像についてPerceptual Hashを計算する。

#     Args:
#         row: データフレームの1行（辞書形式）。

#     Returns:
#         Perceptual Hash（文字列）。
#     """
#     # 画像を読み込む（Pillow形式）
#     image = mi.load_image(row["path"], library="pillow")
#     if image is None:
#         return None

#     # 画像を前処理（明るさ補正とコントラスト強調を統合）
#     preprocessed_image = mi.preprocess_image(image, gamma=1.5, clip_limit=2.0, tile_grid_size=(8, 8))

#     # NumPy配列形式の場合はPillow形式に変換
#     if isinstance(preprocessed_image, np.ndarray):
#         preprocessed_image = Image.fromarray(preprocessed_image)

#     # PIL.Imageのグレースケール画像をRGB形式に変換
#     if preprocessed_image.mode != "RGB":
#         preprocessed_image = preprocessed_image.convert("RGB")

#     # 画像をリサイズ
#     resized_image = mi.resize_image(preprocessed_image, max_length=128)
#     if resized_image is None:
#         return None

#     # Perceptual Hashを計算
#     return mi.calculate_perceptual_hash(resized_image)

# # 並列処理でPerceptual Hashを計算
# with ThreadPoolExecutor() as executor:
#     print("Calculating Perceptual Hashes...")
#     phash_results = list(tqdm(executor.map(process_row_for_phash, df.to_dicts()), total=len(df)))

# # 結果をDataFrameに追加
# df = df.with_columns(pl.Series("perceptual_hash", phash_results))

# # Noneの件数を確認
# none_count = mi.count_none_in_column(df, "perceptual_hash")
# print(f"None values in perceptual_hash: {none_count}")

# # 結果を表示
# print(df.head())

# # 保存
# output_parquet_path = "./temp.parquet"
# df.write_parquet(output_parquet_path)

In [5]:
# # ORB特徴量

# # Parquetデータを読み込む
# df = pl.read_parquet("./temp.parquet")

# # 1. ランダムに100例選択
# image_paths = df["path"].sample(1000).to_list()

# # 2. コードブックを生成
# print("Creating codebook...")
# codebook = mi.create_codebook_orb(image_paths, n_clusters=128, max_length=128)  # リサイズを適用

# # 3. ORB特徴量を固定次元化
# def process_row_for_orb(row, codebook):
#     """
#     各画像について、固定次元のORBヒストグラムを計算する。

#     Args:
#         row: データフレームの1行（辞書形式）。
#         codebook: ORB記述子のクラスタ中心（コードブック）。

#     Returns:
#         固定次元のORBヒストグラム（NumPy配列）。
#     """
#     try:
#         # 画像を読み込む
#         image = mi.load_image(row["path"], library="opencv")
#         if image is None:
#             return np.zeros(len(codebook))  # ゼロベクトルを返す

#         # 画像を前処理（明るさ補正とコントラスト強調を統合）
#         preprocessed_image = mi.preprocess_image(image, gamma=1.5, clip_limit=2.0, tile_grid_size=(8, 8))

#         # 画像をリサイズ
#         resized_image = mi.resize_image(preprocessed_image, max_length=128)
#         if resized_image is None:
#             return np.zeros(len(codebook))  # リサイズ失敗時もゼロベクトルを返す

#         # ORB記述子を抽出
#         descriptors = mi.extract_orb_descriptors(resized_image)
#         if descriptors is None:
#             return np.zeros(len(codebook))  # 記述子が取得できなかった場合もゼロベクトルを返す

#         # 固定次元のヒストグラムを計算
#         return mi.compute_histogram_from_descriptors(descriptors, codebook)

#     except Exception as e:
#         print(f"Error processing row with path {row['path']}: {e}")
#         return np.zeros(len(codebook))  # エラー時もゼロベクトルを返す


# # 並列処理でORBヒストグラムを計算
# with ThreadPoolExecutor() as executor:
#     print("Calculating ORB histograms with resized images...")
#     orb_results = list(tqdm(
#         executor.map(lambda row: process_row_for_orb(row, codebook), df.to_dicts()),
#         total=len(df)
#     ))

# # ORBヒストグラム列を追加
# df = df.with_columns(pl.Series("orb_histogram", orb_results))

# # 4. UMAPによる次元削減
# print("Applying UMAP for dimensionality reduction...")
# orb_histograms = df["orb_histogram"].to_list()  # リスト形式で取得
# orb_histogram_umap = mi.umap_reduction(orb_histograms, n_components=32)

# # 次元削減結果を追加
# df = df.with_columns(pl.Series("orb_histogram_umap", orb_histogram_umap.tolist()))

# # 標準化
# normalized_features = mi.normalize_zscore(df["orb_histogram_umap"].to_list())
# df = df.with_columns(pl.Series("orb_histogram_std", normalized_features.tolist()))

# # 5. 列にNoneが含まれていないか確認
# none_count = mi.count_none_in_column(df, "orb_histogram_std")
# print(f"None values in 'orb_histogram_std': {none_count}")

# # 6. 次元が一致しているか確認
# mi.check_dimensions(df, "orb_histogram_std")

# # 7. 結果を確認
# print(df.head())

# # 8. 保存
# df.write_parquet(output_parquet_path)
# print(f"Processed DataFrame saved to {output_parquet_path}")

In [6]:
# df = pl.read_parquet("./temp.parquet")

# # エッジ特徴量を並列処理で計算
# def process_row_for_edge_histogram(row):
#     """
#     各画像についてエッジヒストグラムを計算する。

#     Args:
#         row: データフレームの1行（辞書形式）。

#     Returns:
#         エッジヒストグラム（リスト）。
#     """
#     # 画像を読み込む（OpenCV形式）
#     image = mi.load_image(row["path"], library="opencv")
#     if image is None:
#         return None

#     # 画像を前処理（明るさ補正とコントラスト強調を統合）
#     preprocessed_image = mi.preprocess_image(image, gamma=1.5, clip_limit=2.0, tile_grid_size=(8, 8))

#     # 画像をリサイズ
#     resized_image = mi.resize_image(preprocessed_image, max_length=128)
#     if resized_image is None:
#         return None

#     # エッジを検出
#     edges = mi.detect_edges(resized_image, method="canny", threshold1=50, threshold2=150)
#     if edges is None:
#         return None

#     # エッジヒストグラムを計算
#     return mi.calculate_edge_histogram(edges, bins=128, range=(0, 256))

# with ThreadPoolExecutor() as executor:
#     print("Calculating edge histograms...")
#     edge_results = list(tqdm(executor.map(process_row_for_edge_histogram, df.to_dicts()), total=len(df)))

# # エッジヒストグラム列を追加
# df = df.with_columns(pl.Series("edge_histogram", edge_results))

# # PCAで次元削減
# pca_features = mi.pca_reduction(df["edge_histogram"].to_list(), n_components=32)
# df = df.with_columns(pl.Series("edge_histogram_pca", pca_features.tolist()))

# # 標準化
# normalized_features = mi.normalize_zscore(df["edge_histogram_pca"].to_list())
# df = df.with_columns(pl.Series("edge_histogram_std", normalized_features.tolist()))

# # Noneや次元の確認
# none_count = mi.count_none_in_column(df, "edge_histogram_std")
# print(f"None values in edge_histogram_std: {none_count}")
# mi.check_dimensions(df, "edge_histogram_std")

# # 結果を保存
# output_parquet_path = "./temp.parquet"
# df.write_parquet(output_parquet_path)
# print(f"Processed DataFrame saved to {output_parquet_path}")

In [7]:
# # データフレームの読み込み
# df = pl.read_parquet("./temp.parquet")

# # 平均輝度と分散輝度を並列処理で計算
# def process_row_for_brightness(row):
#     """
#     各画像について平均輝度と分散輝度を計算する。

#     Args:
#         row: データフレームの1行（辞書形式）。

#     Returns:
#         平均輝度と分散輝度（タプル）。
#     """
#     # 画像を読み込む（OpenCV形式）
#     image = mi.load_image(row["path"], library="opencv")
#     if image is None:
#         return None, None

#     # 画像を前処理（明るさ補正とコントラスト強調を統合）
#     preprocessed_image = mi.preprocess_image(image, gamma=1.5, clip_limit=2.0, tile_grid_size=(8, 8))

#     # 画像をリサイズ
#     resized_image = mi.resize_image(preprocessed_image, max_length=128)
#     if resized_image is None:
#         return None, None

#     # 平均輝度と分散輝度を計算
#     mean_brightness, variance_brightness = mi.calculate_brightness_stats(resized_image)
#     return mean_brightness, variance_brightness


# # 並列処理で平均輝度と分散輝度を計算
# with ThreadPoolExecutor() as executor:
#     print("Calculating brightness features...")
#     brightness_results = list(tqdm(executor.map(process_row_for_brightness, df.to_dicts()), total=len(df)))

# # 平均輝度と分散輝度を個別に取得
# mean_brightness_list, variance_brightness_list = zip(*brightness_results)

# # データフレームに列を追加
# df = df.with_columns(
#     pl.Series("mean_brightness", mean_brightness_list),
#     pl.Series("variance_brightness", variance_brightness_list),
# )

# # Noneが含まれているか確認
# none_count_mean = mi.count_none_in_column(df, "mean_brightness")
# none_count_variance = mi.count_none_in_column(df, "variance_brightness")
# print(f"None values in mean_brightness: {none_count_mean}")
# print(f"None values in variance_brightness: {none_count_variance}")

# # Zスコア正規化を適用（variance_brightness のみ）
# print("Normalizing variance_brightness using Z-score normalization...")
# variance_brightness_normalized = mi.normalize_zscore(df["variance_brightness"].to_list())
# df = df.with_columns(pl.Series("variance_brightness_std", variance_brightness_normalized.tolist()))

# # Noneが含まれているか確認
# none_count_std = mi.count_none_in_column(df, "variance_brightness_std")
# print(f"None values in variance_brightness_std: {none_count_std}")

# # 結果を確認
# print(df.head())

# # 保存
# output_parquet_path = "./temp.parquet"
# df.write_parquet(output_parquet_path)
# print(f"Processed DataFrame saved to {output_parquet_path}")

In [8]:
# # Parquetデータを読み込む
# df = pl.read_parquet("./temp.parquet")

# # GLCM特徴量を並列処理で計算
# def process_row_for_glcm(row):
#     """
#     各画像についてGLCM特徴量を計算する。

#     Args:
#         row: データフレームの1行（辞書形式）。

#     Returns:
#         GLCM特徴量（NumPy配列）。
#     """
#     # 画像を読み込む（OpenCV形式）
#     image = mi.load_image(row["path"], library="opencv")
#     if image is None:
#         return None

#     # 画像を前処理（明るさ補正とコントラスト強調を統合）
#     preprocessed_image = mi.preprocess_image(image, gamma=1.5, clip_limit=2.0, tile_grid_size=(8, 8))

#     # グレースケール変換（必要な場合のみ）
#     if preprocessed_image.ndim == 3:  # カラー画像の場合
#         gray_image = cv2.cvtColor(preprocessed_image, cv2.COLOR_BGR2GRAY)
#     else:  # すでにグレースケールの場合
#         gray_image = preprocessed_image

#     # 画像をリサイズ
#     resized_image = mi.resize_image(gray_image, max_length=128)
#     if resized_image is None:
#         return None

#     # GLCM特徴量を計算
#     return mi.calculate_glcm_features(resized_image)

# # 並列処理でGLCM特徴量を計算
# with ThreadPoolExecutor() as executor:
#     print("Calculating GLCM features...")
#     glcm_results = list(tqdm(executor.map(process_row_for_glcm, df.to_dicts()), total=len(df)))

# # データフレームに列を追加
# df = df.with_columns(pl.Series("glcm_features", glcm_results))

# # Noneが含まれているか確認
# none_count = mi.count_none_in_column(df, "glcm_features")
# print(f"None values in glcm_features: {none_count}")

# # 次元が一致しているか確認
# mi.check_dimensions(df, "glcm_features")

# # GLCM特徴量をZスコア正規化
# print("Normalizing GLCM features using Z-score normalization...")
# glcm_normalized = mi.normalize_zscore(df["glcm_features"].to_list())
# df = df.with_columns(pl.Series("glcm_features_std", glcm_normalized.tolist()))

# # 正規化後の列にNoneが含まれていないか確認
# none_count_std = mi.count_none_in_column(df, "glcm_features_std")
# print(f"None values in glcm_features_std: {none_count_std}")

# # 結果を確認
# print(df.head())

# # 保存
# output_parquet_path = "./temp_with_glcm.parquet"
# df.write_parquet(output_parquet_path)
# print(f"Processed DataFrame saved to {output_parquet_path}")

In [9]:
# # データフレームの読み込み
# df = pl.read_parquet("./temp.parquet")

# # LBP特徴量を計算
# def process_row_for_lbp(row, radius=1, n_points=8, method="uniform", max_length=128):
#     """
#     各画像についてLBP特徴量を計算する。

#     Args:
#         row: データフレームの1行（辞書形式）。
#         radius: LBPの半径。
#         n_points: LBPの近隣点数。
#         method: LBPの計算方法。
#         max_length: リサイズ時の画像の長辺の最大ピクセル数。

#     Returns:
#         LBP特徴量（NumPy配列）。
#     """
#     # 画像を読み込む（OpenCV形式）
#     image = mi.load_image(row["path"], library="opencv")
#     if image is None:
#         return np.zeros(59)  # 59次元のゼロベクトルを返す

#     # 画像を前処理（明るさ補正とコントラスト強調を統合）
#     preprocessed_image = mi.preprocess_image(image, gamma=1.5, clip_limit=2.0, tile_grid_size=(8, 8))

#     # グレースケール変換が必要か確認
#     if preprocessed_image.ndim == 3:  # カラーチャンネルがある場合
#         gray_image = cv2.cvtColor(preprocessed_image, cv2.COLOR_BGR2GRAY)
#     else:
#         gray_image = preprocessed_image

#     # 画像をリサイズ
#     resized_image = mi.resize_image(gray_image, max_length=max_length)
#     if resized_image is None:
#         return np.zeros(59)  # リサイズ失敗時もゼロベクトルを返す

#     # LBP特徴量を計算
#     return mi.calculate_lbp_histogram(resized_image, radius=radius, n_points=n_points, method=method)

# # 並列処理でLBP特徴量を計算
# with ThreadPoolExecutor() as executor:
#     print("Calculating LBP features...")
#     lbp_results = list(tqdm(
#         executor.map(lambda row: process_row_for_lbp(row), df.to_dicts()),
#         total=len(df)
#     ))

# # データフレームに列を追加
# df = df.with_columns(pl.Series("lbp_features", lbp_results))

# # 次元削減（UMAPで32次元へ）
# print("Reducing LBP features using UMAP...")
# lbp_features_umap = mi.umap_reduction(lbp_features_normalized, n_components=32)
# df = df.with_columns(pl.Series("lbp_features_umap", lbp_features_umap.tolist()))

# # Zスコア正規化を適用
# print("Normalizing LBP features using Z-score normalization...")
# lbp_features_normalized = mi.normalize_zscore(df["lbp_features_umap"].to_list())
# df = df.with_columns(pl.Series("lbp_features_std", lbp_features_normalized.tolist()))

# # Noneが含まれているか確認
# none_count = mi.count_none_in_column(df, "lbp_features_std")
# print(f"None values in lbp_features_umap: {none_count}")

# # 次元が一致しているか確認
# mi.check_dimensions(df, "lbp_features_std")

# # データフレームの確認
# print(df.head())

# # 保存
# output_parquet_path = "./processed_with_lbp.parquet"
# df.write_parquet(output_parquet_path)
# print(f"Processed DataFrame saved to {output_parquet_path}")

In [10]:
# # データフレームの読み込み
# df = pl.read_parquet("./temp.parquet")

# # Gaborフィルタ特徴量を計算する関数
# def process_row_for_gabor(row, frequencies, thetas, max_length=128):
#     """
#     各画像についてGaborフィルタ特徴量を計算する。

#     Args:
#         row: データフレームの1行（辞書形式）。
#         frequencies: List of frequencies for the Gabor filters.
#         thetas: List of orientations (angles in radians) for the Gabor filters.
#         max_length: リサイズ時の画像の長辺の最大ピクセル数。

#     Returns:
#         Gaborフィルタ特徴量（NumPy配列）。
#     """
#     # 画像を読み込む（OpenCV形式）
#     image = mi.load_image(row["path"], library="opencv")
#     if image is None:
#         return None

#     # 画像を前処理（明るさ補正とコントラスト強調を統合）
#     preprocessed_image = mi.preprocess_image(image, gamma=1.5, clip_limit=2.0, tile_grid_size=(8, 8))

#     # グレースケール変換
#     if preprocessed_image.ndim == 3:  # カラー画像の場合
#         gray_image = cv2.cvtColor(preprocessed_image, cv2.COLOR_BGR2GRAY)
#     else:
#         gray_image = preprocessed_image

#     # 画像をリサイズ
#     resized_image = mi.resize_image(gray_image, max_length=max_length)
#     if resized_image is None:
#         return None

#     # Gaborフィルタ特徴量を計算
#     return mi.apply_gabor_filters(resized_image, frequencies=frequencies, thetas=thetas)

# # パラメータ設定
# frequencies = [0.1, 0.2, 0.3, 0.4, 0.5]  # Example frequencies
# thetas = [0, np.pi / 4, np.pi / 2, 3 * np.pi / 4]  # Example orientations

# # 並列処理でGaborフィルタ特徴量を計算
# with ThreadPoolExecutor() as executor:
#     print("Calculating Gabor features...")
#     gabor_results = list(tqdm(
#         executor.map(lambda row: process_row_for_gabor(row, frequencies, thetas), df.to_dicts()),
#         total=len(df)
#     ))

# # Gaborフィルタ特徴量列を追加
# df = df.with_columns(pl.Series("gabor_features", gabor_results))

# # Noneが含まれているか確認
# none_count = mi.count_none_in_column(df, "gabor_features")
# print(f"None values in gabor_features: {none_count}")

# # 次元が一致しているか確認
# mi.check_dimensions(df, "gabor_features")

# # Gabor特徴量をZスコア正規化
# print("Normalizing Gabor features using Z-score normalization...")
# gabor_features_normalized = mi.normalize_zscore(df["gabor_features"].to_list())
# df = df.with_columns(pl.Series("gabor_features_std", gabor_features_normalized.tolist()))

# # Noneが含まれているか確認
# none_count_std = mi.count_none_in_column(df, "gabor_features_std")
# print(f"None values in gabor_features_std: {none_count_std}")

# # 結果を確認
# print(df.head())

# # 保存
# output_parquet_path = "./temp_with_gabor.parquet"
# df.write_parquet(output_parquet_path)
# print(f"Processed DataFrame saved to {output_parquet_path}")

In [11]:
# # データフレームを読み込む
# df = pl.read_parquet("./temp.parquet")

# # HOG特徴量を計算する関数
# def process_row_for_hog(row, max_length=128, target_length=5000):
#     """
#     各画像についてHOG特徴量を計算し、次元を揃える。

#     Args:
#         row: データフレームの1行（辞書形式）。
#         max_length: リサイズ時の画像の長辺の最大ピクセル数。
#         target_length: HOG特徴量のターゲット次元数。

#     Returns:
#         次元を揃えたHOG特徴量（NumPy配列）。
#     """
#     # 画像を読み込む
#     image = mi.load_image(row["path"], library="opencv")
#     if image is None:
#         return np.zeros(target_length)  # ゼロベクトルを返す

#     # 画像を前処理
#     preprocessed_image = mi.preprocess_image(image, gamma=1.5, clip_limit=2.0, tile_grid_size=(8, 8))

#     # グレースケール変換
#     if preprocessed_image.ndim == 3:
#         gray_image = cv2.cvtColor(preprocessed_image, cv2.COLOR_BGR2GRAY)
#     else:
#         gray_image = preprocessed_image

#     # リサイズ
#     resized_image = mi.resize_image(gray_image, max_length=max_length)
#     if resized_image is None:
#         return np.zeros(target_length)  # ゼロベクトルを返す

#     # HOG特徴量を計算
#     hog_features = mi.calculate_hog_features(resized_image)

#     # 次元を揃える（ゼロパディングまたはクリッピング）
#     if len(hog_features) < target_length:
#         hog_features = np.pad(hog_features, (0, target_length - len(hog_features)), mode="constant")
#     elif len(hog_features) > target_length:
#         hog_features = hog_features[:target_length]

#     return hog_features

# # 並列処理でHOG特徴量を計算
# with ThreadPoolExecutor() as executor:
#     print("Calculating HOG features...")
#     hog_results = list(tqdm(
#         executor.map(lambda row: process_row_for_hog(row), df.to_dicts()),
#         total=len(df)
#     ))

# # データフレームに列を追加
# df = df.with_columns(pl.Series("hog_features", hog_results))

# # Noneが含まれているか確認
# none_count = mi.count_none_in_column(df, "hog_features")
# print(f"None values in hog_features: {none_count}")

# # 次元が一致しているか確認
# print("Checking dimensions of HOG features...")
# mi.check_dimensions(df, "hog_features")

# # UMAPによる次元削減
# print("Applying UMAP for dimensionality reduction...")
# hog_features_reduced = mi.umap_reduction(df["hog_features"].to_list(), n_components=128)
# df = df.with_columns(pl.Series("hog_features_umap", hog_features_reduced.tolist()))

# # Zスコア正規化
# print("Normalizing HOG features (UMAP) using Z-score normalization...")
# hog_features_normalized = mi.normalize_zscore(df["hog_features_umap"].to_list())
# df = df.with_columns(pl.Series("hog_features_umap_std", hog_features_normalized.tolist()))

# # Noneが含まれているか確認
# none_count_umap = mi.count_none_in_column(df, "hog_features_umap_std")
# print(f"None values in hog_features_umap_std: {none_count_umap}")

# # ゼロベクトルが含まれているか確認
# zero_count_umap = mi.count_zeros_in_column(df, "hog_features_umap_std")
# print(f"Zero vectors in hog_features_umap_std: {zero_count_umap}")

# # 次元が一致しているか確認
# mi.check_dimensions(df, "hog_features_umap_std")

# # 結果を確認
# print(df.head())

# # 保存
# output_parquet_path = "./temp_with_hog_umap.parquet"
# df.write_parquet(output_parquet_path)
# print(f"Processed DataFrame saved to {output_parquet_path}")

In [13]:
# データフレームを読み込む
df = pl.read_parquet("./temp.parquet")

# Zernike Moments を計算する関数
def process_row_for_zernike(row, radius=64, degree=8):
    """
    各画像についてZernike Momentsを計算する。

    Args:
        row: データフレームの1行（辞書形式）。
        radius: 半径のサイズ。
        degree: 最大次数。

    Returns:
        Zernike Momentsのベクトル。
    """
    # 画像を読み込む
    image = mi.load_image(row["path"], library="opencv")
    if image is None:
        return None

    # 画像を前処理
    preprocessed_image = mi.preprocess_image(image, gamma=1.5, clip_limit=2.0, tile_grid_size=(8, 8))

    # グレースケール変換
    if preprocessed_image.ndim == 3:
        gray_image = cv2.cvtColor(preprocessed_image, cv2.COLOR_BGR2GRAY)
    else:
        gray_image = preprocessed_image

    # Zernike Momentsを計算
    zernike_moments = mi.calculate_zernike_moments(gray_image, radius=radius, degree=degree)
    return zernike_moments

# 並列処理でZernike Momentsを計算
with ThreadPoolExecutor() as executor:
    print("Calculating Zernike Moments...")
    zernike_results = list(tqdm(
        executor.map(lambda row: process_row_for_zernike(row), df.to_dicts()),
        total=len(df)
    ))

# データフレームに列を追加
df = df.with_columns(pl.Series("zernike_features", zernike_results))

# None が含まれているか確認
none_count = mi.count_none_in_column(df, "zernike_features")
print(f"None values in zernike_features: {none_count}")

# 次元が一致しているか確認
print("Checking dimensions of Zernike features...")
mi.check_dimensions(df, "zernike_features")

# UMAP による次元削減
print("Reducing Zernike features using UMAP...")
zernike_umap = mi.umap_reduction(df["zernike_features"].to_list(), n_components=32)
df = df.with_columns(pl.Series("zernike_features_umap", zernike_umap.tolist()))

# 正規化
print("Normalizing Zernike features...")
zernike_normalized = mi.normalize_zscore(df["zernike_features_umap"].to_list())
df = df.with_columns(pl.Series("zernike_features_std", zernike_normalized.tolist()))

# None の有無を確認
none_count_std = mi.count_none_in_column(df, "zernike_features_std")
print(f"None values in zernike_features_std: {none_count_std}")

# 結果を確認
print(df.head())

# 保存
output_parquet_path = "./temp_with_zernike.parquet"
df.write_parquet(output_parquet_path)
print(f"Processed DataFrame saved to {output_parquet_path}")

Calculating Zernike Moments...


100%|█████████████████████████████████████████████████████████| 14890/14890 [01:39<00:00, 150.23it/s]


None values in zernike_features: 0
Checking dimensions of Zernike features...
Total rows in zernike_features: 14890
Number of rows with dimension mismatch in zernike_features: 0
Reducing Zernike features using UMAP...
Normalizing Zernike features...
None values in zernike_features_std: 0
shape: (5, 35)
┌───────┬─────────┬─────────────┬───────┬───┬─────────────┬─────────────┬─────────────┬────────────┐
│ index ┆ dirname ┆ path        ┆ width ┆ … ┆ hog_feature ┆ zernike_fea ┆ zernike_fea ┆ zernike_fe │
│ ---   ┆ ---     ┆ ---         ┆ ---   ┆   ┆ s_umap_std  ┆ tures       ┆ tures_umap  ┆ atures_std │
│ i64   ┆ str     ┆ str         ┆ i64   ┆   ┆ ---         ┆ ---         ┆ ---         ┆ ---        │
│       ┆         ┆             ┆       ┆   ┆ list[f64]   ┆ array[f64,  ┆ list[f64]   ┆ list[f64]  │
│       ┆         ┆             ┆       ┆   ┆             ┆ 25]         ┆             ┆            │
╞═══════╪═════════╪═════════════╪═══════╪═══╪═════════════╪═════════════╪═════════════╪═══